### Desenvolvendo o Modelo de Série Temporal

O objetivo aqui é analisar a nossa série temporal para podermos ajustar um LSTM, e como o modelo é de deep learning, quanto mais dado melhor para nós.

In [11]:
# Libs:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [12]:
# Carregando os dados que baixamos pelo yfinance:
dataset = pd.read_csv(r"C:\Users\lucaa\Desktop\TechChallenge - Fase4\data\raw\dataset_PETR_st.csv")
dataset.head(2)

,Date,Adj Close,Close,High,Low,Open,Volume,Acao
0,2018-01-02,5.312316,16.549999,16.549999,16.190001,16.190001,33461800,PETR4
1,2018-01-03,5.360464,16.700001,16.719999,16.370001,16.490000,55940900,PETR4


In [13]:
# Describe da nossa dataset:
dataset.describe()

,Adj Close,Close,High,Low,Open,Volume
count,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1.656000e+03
mean,14.627233,27.563744,27.969674,27.156824,27.566727,6.591767e+07
std,9.404082,5.996867,6.001735,5.970058,5.974129,3.587860e+07
min,3.906060,11.290000,12.180000,10.850000,11.070000,0.000000e+00
25%,8.185170,23.495000,23.922500,23.127500,23.500000,4.292908e+07
50%,10.089991,27.180000,27.585000,26.820000,27.224999,5.785545e+07
75%,17.990360,30.832500,31.232500,30.342500,30.879999,7.933302e+07
max,39.599998,42.900002,42.939999,42.560001,42.770000,4.902304e+08


In [14]:
# Vamos plotar nossa série:

# Selecionando datas especificas para plotar:
datas_mensais = pd.to_datetime(pd.date_range(start = dataset["Date"].min(), end = dataset["Date"].max(), freq = 'Y'))

# Plotando a série temporal
plt.figure(figsize=(12, 4))
sns.lineplot(x = 'Date', y='Adj Close', data = dataset.assign(Date = lambda df_: pd.to_datetime(df_.Date)))
plt.xticks(datas_mensais, rotation = 45)
plt.title('Série Temporal do Fechamento das Ações da Petrobras', fontsize = 13)
plt.xlabel('Data', fontsize = 12)
plt.ylabel('Fechamento (R$)', fontsize = 12)
plt.show()

C:\Users\lucaa\AppData\Local\Temp\ipykernel_16156\2122471645.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  datas_mensais = pd.to_datetime(pd.date_range(start = dataset["Date"].min(), end = dataset["Date"].max(), freq = 'Y'))


ModuleNotFoundError: No module named 'matplotlib_inline'

Comentando a série acima, podemos ver que não necessariamente podemos usar só uma parte para modelagem e predição, visto que o comportamento é linear ao longo do tempo, isto é, se usassemos somente os dados de 2022 pra trás, teriamos péssimas predições para dias atuais, onde o preço médio da ação aumentou significativamente. Dessa forma, importante utilizarmos um treinamento de até dias atuais, separando os últimos 60 dias, por exemplo, para validação do nosso modelo.

### Modelando a ST com LSTM

In [5]:
# Libs para modelagem:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [6]:
# Normalização dos dados
data = dataset['Adj Close'].values.reshape(-1, 1)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Função para criar sequências
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Parâmetros
sequence_length = 10
split_ratio = 0.8

NameError: name 'dataset' is not defined

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import yfinance as yf
from datetime import datetime, timedelta

# Download dos dados
symbol = 'AAPL'
end_date = datetime.now()
start_date = end_date - timedelta(days=1000)
df = yf.download(symbol, start=start_date, end=end_date)
data = df['Close'].values.reshape(-1, 1)

# Normalização dos dados
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Função para criar sequências
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Parâmetros
sequence_length = 10
split_ratio = 0.8

# Criar sequências
X, y = create_sequences(data_scaled, sequence_length)

# Dividir dados em treino e teste
train_size = int(len(X) * split_ratio)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Construir modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1), return_sequences=True),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Treinar modelo
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Fazer previsões
y_pred = model.predict(X_test)

# Reverter normalização
y_test_orig = scaler.inverse_transform(y_test)
y_pred_orig = scaler.inverse_transform(y_pred)

# Calcular métricas
mae = mean_absolute_error(y_test_orig, y_pred_orig)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
mape = np.mean(np.abs((y_test_orig - y_pred_orig) / y_test_orig)) * 100

print(f'\nMétricas de Avaliação:')
print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'MAPE: {mape:.2f}%')

# Plotar resultados
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_test_orig, label='Real')
plt.plot(y_pred_orig, label='Previsto')
plt.title('Previsão vs Valores Reais')
plt.xlabel('Período')
plt.ylabel('Preço')
plt.legend()
plt.show()

# Plotar histórico de perda
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Histórico de Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.show()